In [1]:
import sys
sys.path.append("../")
import warnings
warnings.filterwarnings('ignore')
import numpy as np
import pandas as pd
import os
import argparse
import random
import tqdm
from aif360.metrics import BinaryLabelDatasetMetric
from aif360.metrics import ClassificationMetric
from sklearn.preprocessing import StandardScaler

from aif360.datasets import AdultDataset, GermanDataset, CompasDataset
from utility import get_data,write_to_file
from sklearn.base import clone
from sklearn import tree
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import GradientBoostingClassifier
import copy
from sklearn.tree._tree import TREE_LEAF
import tqdm
from tensorflow import keras

In [2]:
dataset_used = "adult" # "adult", "german", "compas" ,"meps19"
attr = "race"
trials = 50
dataset_orig, privileged_groups,unprivileged_groups,optim_options = get_data(dataset_used, attr)

In [3]:
def get_metrics(clf,test,test_pred,unprivileged_groups,privileged_groups):
    pred = clf.predict(test.features).reshape(-1,1)
    pred = (pred > 0.5).astype("int32")
    #dataset_orig_test_pred = test.copy(deepcopy=True)
    test_pred.labels = pred
    class_metric = ClassificationMetric(test, test_pred,
                         unprivileged_groups=unprivileged_groups, privileged_groups=privileged_groups)
    stat = abs(class_metric.statistical_parity_difference())
    aod = abs(class_metric.average_abs_odds_difference())
    disp = abs(1-class_metric.disparate_impact())
    eod = abs(class_metric.equal_opportunity_difference())
    acc = class_metric.accuracy()
    return acc,stat,disp,aod,eod

In [4]:
data = []
for t in tqdm.tqdm(range(trials)):
    np.random.seed(t)
    dataset_orig_train, dataset_orig_test = dataset_orig.split([0.7], shuffle=True)
    dataset_orig_test,dataset_orig_valid = dataset_orig_test.split([0.5], shuffle=True)
    dataset_orig_test_pred = dataset_orig_test.copy(deepcopy=True)
    dataset_orig_train_pred = dataset_orig_train.copy(deepcopy=True)
    dataset_orig_valid_pred = dataset_orig_valid.copy(deepcopy=True)
    clf = keras.Sequential([
            keras.layers.Dense(64, activation="relu", input_shape=dataset_orig_train.features.shape[1:]),
            keras.layers.Dense(32, activation="relu"),
            keras.layers.Dense(16, activation="relu"),
            keras.layers.Dense(8, activation="relu"),
            keras.layers.Dense(4, activation="relu"),
            keras.layers.Dense(1, activation="sigmoid")
        ])
    clf.compile(loss="binary_crossentropy", optimizer="nadam",metrics=["accuracy"])
    clf.fit(dataset_orig_train.features, dataset_orig_train.labels, epochs=20)
    #train_acc,train_stat,train_disp,train_aod,train_eod = get_metrics(clf,dataset_orig_train,dataset_orig_train_pred,unprivileged_groups,privileged_groups)
    test_acc,test_stat,test_disp,test_aod,test_eod = get_metrics(clf,dataset_orig_test,dataset_orig_test_pred,unprivileged_groups,privileged_groups)
    #valid_acc,valid_stat,valid_disp,valid_aod,valid_eod = get_metrics(clf,dataset_orig_valid,dataset_orig_valid_pred,unprivileged_groups,privileged_groups)
    data.append([test_acc,test_stat,test_disp,test_aod,test_eod])
data = np.array(data)
np.mean(data,axis = 0)

  0%|                                                                                           | 0/50 [00:00<?, ?it/s]

Epoch 1/20
990/990 [==============================] - 4s 2ms/step - loss: 0.4957 - accuracy: 0.8112
Epoch 2/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3734 - accuracy: 0.8231
Epoch 3/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3671 - accuracy: 0.8314
Epoch 4/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3498 - accuracy: 0.8305
Epoch 5/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3455 - accuracy: 0.8351
Epoch 6/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3420 - accuracy: 0.8356
Epoch 7/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3405 - accuracy: 0.8360
Epoch 8/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3393 - accuracy: 0.8373
Epoch 9/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3375 - accuracy: 0.8381
Epoch 10/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3368 - accuracy: 0.8398

  2%|█▋                                                                                 | 1/50 [00:34<28:25, 34.80s/it]

Epoch 1/20
990/990 [==============================] - 4s 2ms/step - loss: 0.6217 - accuracy: 0.8093
Epoch 2/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3566 - accuracy: 0.8246
Epoch 3/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3540 - accuracy: 0.8251
Epoch 4/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3512 - accuracy: 0.8279
Epoch 5/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3485 - accuracy: 0.8276
Epoch 6/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3477 - accuracy: 0.8275
Epoch 7/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3563 - accuracy: 0.8190
Epoch 8/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3677 - accuracy: 0.8117
Epoch 9/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3671 - accuracy: 0.8145
Epoch 10/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3653 - accuracy: 0.8151

  4%|███▎                                                                               | 2/50 [01:10<28:17, 35.37s/it]

...

Epoch 1/20
990/990 [==============================] - 5s 2ms/step - loss: 0.6635 - accuracy: 0.7294
Epoch 2/20
990/990 [==============================] - 2s 2ms/step - loss: 0.5307 - accuracy: 0.7766
Epoch 3/20
990/990 [==============================] - 2s 2ms/step - loss: 0.5034 - accuracy: 0.7927
Epoch 4/20
990/990 [==============================] - 2s 2ms/step - loss: 0.4840 - accuracy: 0.7947
Epoch 5/20
990/990 [==============================] - 2s 2ms/step - loss: 0.4814 - accuracy: 0.7933
Epoch 6/20
990/990 [==============================] - 2s 2ms/step - loss: 0.4809 - accuracy: 0.7951
Epoch 7/20
990/990 [==============================] - 2s 2ms/step - loss: 0.4792 - accuracy: 0.7964
Epoch 8/20
990/990 [==============================] - 2s 2ms/step - loss: 0.4788 - accuracy: 0.7957
Epoch 9/20
990/990 [==============================] - 2s 2ms/step - loss: 0.4792 - accuracy: 0.7966
Epoch 10/20
990/990 [==============================] - 2s 2ms/step - loss: 0.4768 - accuracy: 0.7973

 98%|████████████████████████████████████████████████████████████████████████████████▎ | 49/50 [37:21<00:42, 42.09s/it]

Epoch 1/20
990/990 [==============================] - 4s 2ms/step - loss: 0.4910 - accuracy: 0.8054
Epoch 2/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3661 - accuracy: 0.8145
Epoch 3/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3570 - accuracy: 0.8173
Epoch 4/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3545 - accuracy: 0.8204
Epoch 5/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3530 - accuracy: 0.8206
Epoch 6/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3518 - accuracy: 0.8213
Epoch 7/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3490 - accuracy: 0.8241
Epoch 8/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3484 - accuracy: 0.8235
Epoch 9/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3473 - accuracy: 0.8243
Epoch 10/20
990/990 [==============================] - 2s 2ms/step - loss: 0.3466 - accuracy: 0.8285

100%|██████████████████████████████████████████████████████████████████████████████████| 50/50 [38:01<00:00, 45.64s/it]


array([0.81026684, 0.08105222,        nan, 0.04020875, 0.0467107 ])

In [6]:
for row in data:
    print (*row)

0.8536045997346307 0.12198927486869156 0.5100023041474655 0.06799098690020525 0.08704197396329949
0.8236768391567153 0.09270265408229239 0.41519941605140653 0.05558584492638622 0.07274576392684651
0.8163054695562435 0.0818356329026152 0.30426877255827833 0.01946797332290482 0.005691093778342204
0.8140940586761021 0.09414127707274567 0.3882410285515996 0.0569023375132331 0.08164829760768322
0.8149786230281586 0.09451581561049338 0.2987374886260237 0.053223247188290364 0.051569437354001324
0.8434321096859797 0.06152747325167385 0.2993033774065539 0.04267260083562857 0.07527606213737603
0.816452896948253 0.08984895518072186 0.3405263717480024 0.045758953948122245 0.04882540875775232
0.7554179566563467 0.0 nan 0.0 0.0
0.8223499926286304 0.09256300360020572 0.33829573934837087 0.06580750753768015 0.10366870483602009
0.7459826035677429 0.0 nan 0.0 0.0
0.8278048061329795 0.10104102507933754 0.3762193353786345 0.055878690138924325 0.07988543334611309
0.8155683325961963 0.13943203556971864 0.45